In [5]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [7]:
%sql sqlite:///my_data1.db

In [8]:
import pandas as pd
df = pd.read_csv("dataset_part_2.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

90

In [9]:
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,1
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,1
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,1
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,1
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,1


In [13]:
#DROP THE TABLE IF EXISTS

%sql DROP TABLE IF EXISTS SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


[]

In [14]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

Display the names of the unique launch sites in the space mission

In [15]:
%%sql
SELECT DISTINCT "LaunchSite"
FROM SPACEXTBL;


 * sqlite:///my_data1.db
Done.


LaunchSite
CCSFS SLC 40
VAFB SLC 4E
KSC LC 39A


Display 5 records where launch sites begin with the string 'CCA'


In [17]:
%%sql
SELECT *
FROM SPACEXTBL
WHERE "LaunchSite" LIKE 'CCSFS%'
LIMIT 5;


 * sqlite:///my_data1.db
Done.


FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
1,2010-06-04,Falcon 9,6123.547647058824,LEO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0003,-80.577366,28.5618571,1
2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0005,-80.577366,28.5618571,1
3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0007,-80.577366,28.5618571,1
5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1004,-80.577366,28.5618571,1
6,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1005,-80.577366,28.5618571,1


Display the total payload mass carried by boosters launched by NASA (CRS)

In [34]:
%%sql
SELECT SUM("PayloadMass") AS Sum
FROM SPACEXTBL
WHERE "BoosterVersion" = 'Falcon 9'
  AND "PayloadMass" IS NOT NULL;


 * sqlite:///my_data1.db
Done.


Sum
551119.2882352942


Display average payload mass carried by booster version F9 v1.1

In [31]:
%%sql
SELECT AVG("PayloadMass") AS AvgPayloadMass
FROM SPACEXTBL
WHERE "BoosterVersion" = 'Falcon 9'
  AND "PayloadMass" IS NOT NULL;


 * sqlite:///my_data1.db
Done.


AvgPayloadMass
6123.547647058824


List the date when the first succesful landing outcome in ground pad was acheived.

In [38]:
%%sql
SELECT MIN("Date") AS FirstSuccessfulGroundPadLandingDate
FROM SPACEXTBL
WHERE "Outcome" = 'True RTLS';


 * sqlite:///my_data1.db
Done.


FirstSuccessfulGroundPadLandingDate
2015-12-22


List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [41]:
%%sql
SELECT "BoosterVersion", "PayloadMass"
FROM SPACEXTBL
WHERE ("Outcome" = 'True ASDS' OR "Outcome" = 'True Ocean')
  AND "PayloadMass" > 4000
  AND "PayloadMass" < 6000;



 * sqlite:///my_data1.db
Done.


BoosterVersion,PayloadMass
Falcon 9,4696.0
Falcon 9,4600.0
Falcon 9,5300.0
Falcon 9,5200.0
Falcon 9,4230.0
Falcon 9,5800.0
Falcon 9,5000.0


List the total number of successful and failure mission outcomes¶

In [42]:
%%sql
SELECT 
  CASE 
    WHEN "Outcome" LIKE 'True%' THEN 'Success'
    WHEN "Outcome" LIKE 'False%' THEN 'Failure'
    ELSE 'Unknown'
  END AS MissionStatus,
  COUNT(*) AS Total
FROM SPACEXTBL
GROUP BY MissionStatus;


 * sqlite:///my_data1.db
Done.


MissionStatus,Total
Failure,9
Success,60
Unknown,21


List all the booster_versions that have carried the maximum payload mass. Use a subquery.¶

In [44]:
%%sql
SELECT "BoosterVersion", "PayloadMass"
FROM SPACEXTBL
WHERE "PayloadMass" = (SELECT MAX("PayloadMass") FROM SPACEXTBL);



 * sqlite:///my_data1.db
Done.


BoosterVersion,PayloadMass
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0
Falcon 9,15600.0


List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [46]:
%%sql
SELECT
  CASE substr("Date", 6, 2)
    WHEN '01' THEN 'January'
    WHEN '02' THEN 'February'
    WHEN '03' THEN 'March'
    WHEN '04' THEN 'April'
    WHEN '05' THEN 'May'
    WHEN '06' THEN 'June'
    WHEN '07' THEN 'July'
    WHEN '08' THEN 'August'
    WHEN '09' THEN 'September'
    WHEN '10' THEN 'October'
    WHEN '11' THEN 'November'
    WHEN '12' THEN 'December'
  END AS MonthName,
  "Outcome",
  "BoosterVersion",
  "LaunchSite"
FROM SPACEXTBL
WHERE substr("Date", 1, 4) = '2015'
  AND ("Outcome" = 'False ASDS' OR "Outcome" = 'False Ocean');


 * sqlite:///my_data1.db
Done.


MonthName,Outcome,BoosterVersion,LaunchSite
January,False ASDS,Falcon 9,CCSFS SLC 40
April,False ASDS,Falcon 9,CCSFS SLC 40


In [47]:
%%sql
SELECT "Outcome", COUNT(*) AS OutcomeCount
FROM SPACEXTBL
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Outcome"
ORDER BY OutcomeCount DESC;


 * sqlite:///my_data1.db
Done.


Outcome,OutcomeCount
None None,9
True ASDS,5
False ASDS,4
True RTLS,3
True Ocean,3
None ASDS,2
False Ocean,2
